**Data Loading**


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("Tech_Layoffs_Tracker.csv")
df.head()

,Company,Location HQ,# Laid Off,Date,%,Industry,Source,Stage,$ Raised (mm),Country,Date Added
0,Cars24,"Gurugram,Non-U.S.",120.0,5/27/2025,NaN,Transportation,https://inc42.com/buzz/cars24-to-fire-120-more...,Series G,$1300,India,5/28/2025
1,nCino,Wilmington,NaN,5/27/2025,7%,Finance,https://www.housingwire.com/articles/ncino-wor...,Post-IPO,$1100,United States,5/29/2025
2,Otipy,"New Delhi,Non-U.S.",300.0,5/23/2025,100%,Food,https://economictimes.indiatimes.com/tech/star...,Series B,$44,India,5/27/2025
3,LeddarTech,"Quebec,Non-U.S.",138.0,5/22/2025,95%,Transportation,https://betakit.com/leddartech-cuts-95-percent...,Post-IPO,$402,Canada,5/23/2025
4,Climeworks,"Zurich,Non-U.S.",106.0,5/21/2025,22%,Energy,https://www.bloomberg.com/news/articles/2025-0...,Private Equity,$734,Switzerland,5/23/2025


**Data Exploration**

In [ ]:
#top rows and the columns of the dataframe
df.head()

,Company,Location HQ,# Laid Off,Date,%,Industry,Source,Stage,$ Raised (mm),Country,Date Added
0,Cars24,"Gurugram,Non-U.S.",120.0,5/27/2025,NaN,Transportation,https://inc42.com/buzz/cars24-to-fire-120-more...,Series G,$1300,India,5/28/2025
1,nCino,Wilmington,NaN,5/27/2025,7%,Finance,https://www.housingwire.com/articles/ncino-wor...,Post-IPO,$1100,United States,5/29/2025
2,Otipy,"New Delhi,Non-U.S.",300.0,5/23/2025,100%,Food,https://economictimes.indiatimes.com/tech/star...,Series B,$44,India,5/27/2025
3,LeddarTech,"Quebec,Non-U.S.",138.0,5/22/2025,95%,Transportation,https://betakit.com/leddartech-cuts-95-percent...,Post-IPO,$402,Canada,5/23/2025
4,Climeworks,"Zurich,Non-U.S.",106.0,5/21/2025,22%,Energy,https://www.bloomberg.com/news/articles/2025-0...,Private Equity,$734,Switzerland,5/23/2025


In [ ]:
#check for the important informations of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4076 entries, 0 to 4075
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company        4076 non-null   object 
 1   Location HQ    4075 non-null   object 
 2   # Laid Off     2659 non-null   float64
 3   Date           4076 non-null   object 
 4   %              2593 non-null   object 
 5   Industry       4075 non-null   object 
 6   Source         4073 non-null   object 
 7   Stage          4069 non-null   object 
 8   $ Raised (mm)  3629 non-null   object 
 9   Country        4076 non-null   object 
 10  Date Added     4076 non-null   object 
dtypes: float64(1), object(10)
memory usage: 350.4+ KB


This shows the column names, non-null value counts and the data types of the column values.

In [ ]:
# Stats for numeric columns
df.describe()

,# Laid Off
count,2659.000000
mean,278.851448
std,962.059729
min,3.000000
25%,40.000000
50%,85.000000
75%,200.000000
max,22000.000000


This shows the basic statistical inoformations of the numerical columns of the dataset.

In [ ]:
# List of column names
df.columns

Index(['Company', 'Location HQ', '# Laid Off', 'Date', '%', 'Industry',
       'Source', 'Stage', '$ Raised (mm)', 'Country', 'Date Added'],
      dtype='object')

This shows the column names of the dataset.

In [ ]:
# check for the unique values in each columns
df.nunique()

,0
Company,2798
Location HQ,264
# Laid Off,364
Date,1044
%,73
Industry,30
Source,3827
Stage,16
$ Raised (mm),715
Country,66


This shows the total number of the unique values in each columns of the dataset.

**Data Cleaning**


In [ ]:
# Check for the missing values in the dataset
df.isnull().sum()      #check for the total number of null values in each columns

,0
Company,0
Location HQ,1
# Laid Off,1417
Date,0
%,1483
Industry,1
Source,3
Stage,7
$ Raised (mm),447
Country,0


This shows that the Laid Off, % and Raised(mm) column has the highest number of missing values.

In [ ]:
def Handle_Missing_Values(df):
    """
    Cleans missing values and formats key columns in the Layoffs dataset.
    """
    # Drop rows with missing Company or Date (essential fields)
    df = df.dropna(subset=["Company", "Date"])

    # Fill categorical columns
    df["Location HQ"] = df["Location HQ"].fillna(df["Location HQ"].mode()[0])
    df["Industry"] = df["Industry"].fillna("Unknown")
    df["Source"] = df["Source"].fillna("Unavailable")
    df["Stage"] = df["Stage"].fillna("Unknown")

    # Convert % column from string to float
    df['%'] = df['%'].astype(str).str.replace('%', '', regex=False)
    df['%'] = pd.to_numeric(df['%'], errors='coerce')

    # Convert '$ Raised (mm)' column from string to float
    df['$ Raised (mm)'] = df['$ Raised (mm)'].astype(str).str.replace('$', '', regex=False)
    df['$ Raised (mm)'] = pd.to_numeric(df['$ Raised (mm)'], errors='coerce')

    # Fill numeric columns
    df['# Laid Off'] = df.groupby('Industry')['# Laid Off'].transform(lambda x: x.fillna(x.median()))
    df['# Laid Off'] = df['# Laid Off'].fillna(df['# Laid Off'].median())

    df['%'] = df.groupby('Industry')['%'].transform(lambda x: x.fillna(x.median()))
    df['%'] = df.groupby('Stage')['%'].transform(lambda x: x.fillna(x.median()))
    df['%'] = df['%'].fillna(df['%'].median())

    df['$ Raised (mm)'] = df.groupby('Stage')['$ Raised (mm)'].transform(lambda x: x.fillna(x.median()))
    df['$ Raised (mm)'] = df['$ Raised (mm)'].fillna(df['$ Raised (mm)'].median())
    return df

df = Handle_Missing_Values(df)

In [ ]:
# CHeck for the missing values after applying handling methods
df.isnull().sum()

,0
Company,0
Location HQ,0
# Laid Off,0
Date,0
%,0
Industry,0
Source,0
Stage,0
$ Raised (mm),0
Country,0


Missing Values handled successfully.


**Duplicate Values**

In [ ]:
# Check for the Duplicate Values in the dataset
df.duplicated().any() # Check if there is any duplicate value in the dataset

np.False_

As the result is false, it means there is not duplicate value in the dataset.

In [ ]:
df[df.duplicated()]

,Company,Location HQ,# Laid Off,Date,%,Industry,Source,Stage,$ Raised (mm),Country,Date Added


It confirms that there is no duplicate row in the dataframe.

In [ ]:
# Format the dates in the same format : dd-mm-yyyy
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Date Added'] = pd.to_datetime(df['Date Added'], errors='coerce')

In [ ]:
# Change the Laid off and % column name for better understanding
df = df.rename(columns={'# Laid Off': 'Number_of_Layoff', '%': 'Layoff_Percentage', '$ Raised (mm)' : 'Raised_Amount($mm)', 'Date': 'Layoff_Date'})
# Remove '%' and '$' sign and convert Layoff_Percentage and Raised_Amount($mm) to numeric for calculations
df['Layoff_Percentage'] = df['Layoff_Percentage'].astype(str).str.replace('%', '', regex=False)
df['Raised_Amount($mm)'] = df['Raised_Amount($mm)'].astype(str).str.replace('$', '', regex=False)
df['Layoff_Percentage'] = pd.to_numeric(df['Layoff_Percentage'], errors='coerce')
df['Raised_Amount($mm)'] = pd.to_numeric(df['Raised_Amount($mm)'], errors='coerce')
df.head()

,Company,Location HQ,Number_of_Layoff,Layoff_Date,Layoff_Percentage,Industry,Source,Stage,Raised_Amount($mm),Country,Date Added
0,Cars24,"Gurugram,Non-U.S.",120.0,2025-05-27,17.0,Transportation,https://inc42.com/buzz/cars24-to-fire-120-more...,Series G,1300.0,India,2025-05-28
1,nCino,Wilmington,70.5,2025-05-27,7.0,Finance,https://www.housingwire.com/articles/ncino-wor...,Post-IPO,1100.0,United States,2025-05-29
2,Otipy,"New Delhi,Non-U.S.",300.0,2025-05-23,100.0,Food,https://economictimes.indiatimes.com/tech/star...,Series B,44.0,India,2025-05-27
3,LeddarTech,"Quebec,Non-U.S.",138.0,2025-05-22,95.0,Transportation,https://betakit.com/leddartech-cuts-95-percent...,Post-IPO,402.0,Canada,2025-05-23
4,Climeworks,"Zurich,Non-U.S.",106.0,2025-05-21,22.0,Energy,https://www.bloomberg.com/news/articles/2025-0...,Private Equity,734.0,Switzerland,2025-05-23


In [ ]:
# remove the Non-U.S. label from the Location HQ column
df['Location HQ'] = df['Location HQ'].str.replace(',? ?Non-U\.S\.', '', regex=True).str.strip()
df.head()

,Company,Location HQ,Number_of_Layoff,Layoff_Date,Layoff_Percentage,Industry,Source,Stage,Raised_Amount($mm),Country,Date Added
0,Cars24,Gurugram,120.0,2025-05-27,17.0,Transportation,https://inc42.com/buzz/cars24-to-fire-120-more...,Series G,1300.0,India,2025-05-28
1,nCino,Wilmington,70.5,2025-05-27,7.0,Finance,https://www.housingwire.com/articles/ncino-wor...,Post-IPO,1100.0,United States,2025-05-29
2,Otipy,New Delhi,300.0,2025-05-23,100.0,Food,https://economictimes.indiatimes.com/tech/star...,Series B,44.0,India,2025-05-27
3,LeddarTech,Quebec,138.0,2025-05-22,95.0,Transportation,https://betakit.com/leddartech-cuts-95-percent...,Post-IPO,402.0,Canada,2025-05-23
4,Climeworks,Zurich,106.0,2025-05-21,22.0,Energy,https://www.bloomberg.com/news/articles/2025-0...,Private Equity,734.0,Switzerland,2025-05-23



**Feature Engineering**

In [ ]:
# Extract the month and year column from the layoff date

# Ensure it's datetime format (even though it already looks like it)
df['Layoff_Date'] = pd.to_datetime(df['Layoff_Date'], errors='coerce')

# Extract full month name (like "April")
df['Layoff_Month'] = df['Layoff_Date'].dt.month

# Extract year (like 2025)
df['Layoff_Year'] = df['Layoff_Date'].dt.year

# Extract Quarter
df['Layoff_Quarter'] = df['Layoff_Date'].dt.quarter
df.head()

,Company,Location HQ,Number_of_Layoff,Layoff_Date,Layoff_Percentage,Industry,Source,Stage,Raised_Amount($mm),Country,Date Added,Layoff_Month,Layoff_Year,Layoff_Quarter
0,Cars24,Gurugram,120.0,2025-05-27,17.0,Transportation,https://inc42.com/buzz/cars24-to-fire-120-more...,Series G,1300.0,India,2025-05-28,5,2025,2
1,nCino,Wilmington,70.5,2025-05-27,7.0,Finance,https://www.housingwire.com/articles/ncino-wor...,Post-IPO,1100.0,United States,2025-05-29,5,2025,2
2,Otipy,New Delhi,300.0,2025-05-23,100.0,Food,https://economictimes.indiatimes.com/tech/star...,Series B,44.0,India,2025-05-27,5,2025,2
3,LeddarTech,Quebec,138.0,2025-05-22,95.0,Transportation,https://betakit.com/leddartech-cuts-95-percent...,Post-IPO,402.0,Canada,2025-05-23,5,2025,2
4,Climeworks,Zurich,106.0,2025-05-21,22.0,Energy,https://www.bloomberg.com/news/articles/2025-0...,Private Equity,734.0,Switzerland,2025-05-23,5,2025,2


In [ ]:
# Delete the source and the Date Added column as they were redundent
df = df.drop(['Source', 'Date Added'], axis=1)

In [ ]:
# Add new derived columns for the simplicity
# Layoff_Severity_Level: (Low < 50, Medium < 500, High >= 500 layoffs)
df['Layoff_Severity'] = pd.cut(df['Number_of_Layoff'],
                               bins=[0, 50, 500, df['Number_of_Layoff'].max()],
                               labels=['Low', 'Medium', 'High'])

# Funding Category (Small < $50M, Mid < $250M, Large > $250M)
# Create Funding_Category column based on Raised_Amount($mm)
df['Funding_Category'] = pd.cut(df['Raised_Amount($mm)'],
                                bins=[-1, 50, 250, df['Raised_Amount($mm)'].max()],
                                labels=['low_funded', 'Medium_funded', 'high_funded'])

In [ ]:
df.duplicated().any() # check for the duplicate rows after the feature engineering

np.True_

In [ ]:
# Check for exact duplicate rows
duplicate_rows = df[df.duplicated()]
print(f"Total duplicate rows: {duplicate_rows.shape[0]}")

Total duplicate rows: 2


In [ ]:
# View duplicate entries
df[df.duplicated()].head()

,Company,Location HQ,Number_of_Layoff,Layoff_Date,Layoff_Percentage,Industry,Stage,Raised_Amount($mm),Country,Layoff_Month,Layoff_Year,Layoff_Quarter,Layoff_Severity,Funding_Category
2598,Beyond Meat,Los Angeles,200.0,2022-10-14,19.0,Food,Post-IPO,122.0,United States,10,2022,4,Medium,Medium_funded
3209,Cazoo,London,750.0,2022-06-07,15.0,Transportation,Post-IPO,2000.0,United Kingdom,6,2022,2,High,high_funded


In [ ]:
df = df.drop_duplicates() # delete the duplicate rows

In [ ]:
df.reset_index(drop=True, inplace=True) # Reset Index After Dropping - This helps clean up the index after row removal.

In [ ]:
df.duplicated().any() # Check again for the duplicate values and confirm

np.False_

In [ ]:
df['Number_of_Layoff'] = df['Number_of_Layoff'].fillna(0).astype(int) # Convert number_of_layoff from float to int type

In [ ]:
df.to_csv('Layoffs_cleaned.csv', index=False)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4074 entries, 0 to 4073
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Company             4074 non-null   object        
 1   Location HQ         4074 non-null   object        
 2   Number_of_Layoff    4074 non-null   int64         
 3   Layoff_Date         4074 non-null   datetime64[ns]
 4   Layoff_Percentage   4074 non-null   float64       
 5   Industry            4074 non-null   object        
 6   Stage               4074 non-null   object        
 7   Raised_Amount($mm)  4074 non-null   float64       
 8   Country             4074 non-null   object        
 9   Layoff_Month        4074 non-null   int32         
 10  Layoff_Year         4074 non-null   int32         
 11  Layoff_Quarter      4074 non-null   int32         
 12  Layoff_Severity     4074 non-null   category      
 13  Funding_Category    4074 non-null   category    